# Library

In [ ]:
!pip install openai

In [216]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from pydantic import BaseModel
from openai import OpenAI

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans, DBSCAN

# Large Language Model (LLM)

방대한 양의 텍스트 데이터를 기반으로 훈련된 인공지능 모델. <br>
파라미터 수에 따라 LLM을 분류하나 절대적인 기준은 존재하지 않음. <br>
&nbsp;&nbsp;&nbsp;&nbsp;한 예로 1000억개(100B) 미만이면 sLLM, 크면 LLM으로 분류. <br>

현재 크게 다섯 가지 모델 사용

|model|개발사|특징|
|-----|----|-----|
|Claude|Anthropic|텍스트 생성, 분석, 코딩 지원, 질문 답변 등 다양한 작업 수행|
|Gemini|Goodle|멀티모달 능력을 강조|
|GPT|OpenAI|텍스트 생성, 번역, 요약, 질문 답변 등 다양한 자연어 처리 작업|
|LLaMA|Meta|연구 커뮤니티를 위해 설계|
|Mistral|Mistral|효율적인 LLM을 목표로 개발된 모델로, 경량화된 구조를 통해 높은 성능과 빠른 처리 속도를 제공|

## GPT

Playground: https://platform.openai.com/playground/ <br>
API: https://openai.com/index/openai-api/ <br>
Documentation: https://platform.openai.com/docs/overview <br>

### paper

#### GPT1

paper: [Improving Language Understanding by Generative Pre-Training](https://cdn.openai.com/research-covers/language-unsupervised/language_understanding_paper.pdf)

![](https://ffighting.net/wp-content/uploads/2023/09/image-26.png)

##### 문제 상황

기존 deep learning 모델에서는 방대한 양의 labled data를 활용하여 학습하기에, label data를 갖는 데이터 취득이 어려운 경우 적용이 제한됨. <br>
- unlabeled data를 모델이 활용하게 하는 것이, label data를 수집하는 것보다 경제적
- supervised의 경우에도 unsupervised로 잘 학습 되면 더 좋은 performance 달성 가능

<br>
기존 unlabeled data는 단어 수준의 정보 밖에 얻지 못함
- 어떻게 optimization하는 것이 효과적인지 불분명
- 어떠한 fine-tuning이 가장 효과적인지 알 수 없음

##### Contribution

semi-supervised 학습 방법 제안 <br>
- unsupervised: pre-training
- supervised: fine-tuning

##### 방법

###### Unsupervised pre-training

Tokens $U = u_1, u_2, ..., u_n$에서 likelihood가 maximize하도록 학습

$$
L_1(U) = \Sigma_i \log P(u_i|u_{i-k}, ..., u_{i-1}; \theta) \\
\text{where  } k: \text{context window,  } \theta: \text{weights}
$$

transformer의 decoder layer에서 encoder-decoder attention이 encoder가 없어 제외 <br>

###### Supervised fine-tuning

last hidden state $h_l^m$을 y값 예측을 위한 linear layer를 통과시킴
$$ P(y|x^1, ..., x^m) = \text{softmax}(h_l^mW_y) $$

<br>

이는 아래의 수식을 maximize하는 것
$$ L_2(C) = \Sigma_{(x, y)} \log P(y|x^1, ..., x^m) $$

<br>

추가적인 task를 통하여 일반화 성능 향상 가능 <br>
auxiliary task: NER, POS tagging, etc.
$$ L_3(C) = L_2(C) + \lambda * L_1(c) $$

##### 한계

unsupervised를 지향했음에도 fine-tuning이 필요. <br>
일부 task에서는 input transformation 필요.

#### GPT2

paper: [Language Models are Unsupervised Multitask Learners](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf)

##### 문제상황

single domain dataset에 대한  single task training이 generalization에 방해되고 있음 <br>
- pre-training과 supervised fine-tuning을 이용하는 방법도 여전히 supervised training을 요구.

##### Cntribution

zero shot setting에서 down stream task가 가능함을 보임

##### 방법

###### conditional distribution change

$ p(output|input) $ -> $ p(output|input, task) $ <br>
-> 동일 input과 output일지라도 task에 따라서 결과가 달라질 수 있음

ex) <br>
translation: (translate to french, english text, french text) <br>
reading comprehension: (answer the question, document, question, answer) <br>

###### training dataset

WebText 셋 구축
- Reddit에서 3 karma이상의 외부 링크 스크랩
    - 4,500만 개 링크의 하위 텍스트 집합 포함
- wikipedia 데이터 제거
- 최종적으로 40G, 약 800만 개의 문서로 구성

###### input representation

BPE를 사용하여 인코딩 <br>
- byte sequence 문자 category 간 병합되지 않도록 설정

###### model

![](https://velog.velcdn.com/images/tm011899/post/e1728f56-9ab0-4d85-9778-731e57c4171d/image.png)

Hyperparameter:
|parameters|layers|$d_{model}$|
|----------|------|---------|
|117M      | 12   | 768     |
|345M      | 24   | 1024     |
|762M      | 36   | 1280     |
|1542M      | 48   | 1600     |

<br>

layer normalization: <br>
LayerNorm(sublayer(x) + x) -> x + sublayer(LayerNorm(x))

weight inittialization: <br>
weight*(1/N), N: residual layer의 수

vocab size: <br>
40478-> 50257

context size: <br>
512 -> 1024

##### 한계

zero-shot, few-shot learning에 대해서는 underfit <br>
특정 task 수행을 위해 task-specific dataset과 fine-tuning 필요


#### GPT3

paper: [Language Models are Few-Shot Learners](https://arxiv.org/pdf/2005.14165)

##### 문제상황

task-specific datasets 및 task-specific fine-tuning의 필요성. <br>
사전 학습 중 방대한 데이터를 학습하지만, fine-tuning 과정에서 작은 task 분포에 대해 fit 됨 <br>
사람은 언어 문제에서 다량의 supervised dataset을 필요로 하지 않음 <br>

##### Contribution

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbcCkzC%2FbtqEzhJ441q%2FCr6nzgvZHP4cDBj6bksKf0%2Fimg.png)

큰 모델이 few-shot setting에서 잘 작동하는지 검증
- few-shot learning (in-context learning): context window 내에 최대한 많은 예제 전달
- one-shot learning: 하나의 예제만 허용
- zero-shot learning: 예제를 사용하지 않고 task에 대한 설명이나 지시사항만 전달

![](https://miro.medium.com/v2/resize:fit:828/format:webp/1*Gbahtp2crhCYYv13PihMWA.jpeg)

##### 방법

###### architecture

GPT2와 유사. <br>
- attention -> sparse attention

###### training dataset

|Dataset|Quantity(tokens) | Weight training mix | Epochs elapsed when training for 300B tokens|
|--|-----------------|---------------------|---------------------------------------------|
|Common Crawl(filtered)|410 billion | 60% | 0.44|
|WebText2           | 19 billion    | 22% | 2.9|
|Books1           | 12 billion    | 8% | 1.9|
|Books2           | 12 billion    | 8% | 0.43|
|Wikipedia           | 3 billion    | 3% | 3.4|


###### model

|model name|$n_{params}$|$n_{layers}$|$d_{model}$|$d_{head}$|batch size|
|----------|------------|------------|-----------|----------|-----|
|GPT3-small|125M        |  12        |  768      |   12     | 0.5M|
|GPT3-Medium|350M        |  24        |  1024      |   16     | 0.5M|
|GPT3-Large|760M        |  24        |  1536      |   16     | 0.5M|
|GPT3-XL|1.3B        |  24        |  2048      |   24     | 1M|
|GPT3-2.7B|2.7B        |  32        |  2560      |   32     | 1M|
|GPT3-6.7B|6.7B        |  32        |  4096      |   32     | 2M|
|GPT3-13B|13B        |  40        |  5140      |   40     | 2M|
|GPT3-175B|175B        |  96        |  12288      |   96     | 3.2M|

##### 한계

ANLI데이터와 RACE, QuAC와 같은 QA 셋에서 few-shot learning 성능이 그다지 좋지 않음. <br>
특정 문서에서 특정 단어가 많이 쓰이는 등의 문제. <br>
모델 파라미터의 증가로 추론 등에서도 많은 리소스 소비. <br>

#### 비교

![](https://wikidocs.net/images/page/184363/gpt.PNG)

<br>
<br>

| 모델          | 파라미터 개수        | 디코더의 층 | 처리 가능한 토큰 개수 | 은닉층의 크기 |
|---------------|---------------------|-------------|----------------------|----------------|
| GPT-1        | 1억 1700만개       | 12          | 512                  | 768            |
| GPT-2        | 15억               | 48          | 1024                 | 1600           |
| GPT-3        | 1,750억            | 96          | 2048                 | 12288          |
| GPT-3.5      | ?                   | ?           | ?                    | ?              |
| GPT-4        | 1조 8천억(추정)    | ?           | 128,000              | ?              |


## ChatGPT

GPT3.5를 기반으로 기존 훈련 방법에서 RLHF(Reinforcement Learning from Human Feedback)을 통하여 대화의 품질을 높힘. <br>
정확한 parameter의 수가 공개되지 않음. <br>

Pricing: https://openai.com/api/pricing/ <br>
Playground: https://platform.openai.com/playground/ <br>
API: https://openai.com/index/openai-api/ <br>
Documentation: https://platform.openai.com/docs/overview <br>

### Playground

OpenAI API의 자연어 처리를 손쉽게 실행할 수 있는 웹 UI. <br>
사용하는 모델에 따라 과금. <br>

모드에는 크게 다섯 가지 존재 <br>
1. Chat: 채팅 모드
2. Realtime: 실시간 서비스 모드
3. Assistants: 특정 앱에서 사용할 수 있는 도우미 서비스 모드
4. TTS: 텍스트를 음성으로 변경하는 모드
5. Completions: 텍스트 생성 모드

<br>

사용 시 아래의 파라미터 조절 
1. Temperature: 무작위성 조절 parameter. 답이 있는 경우 0, 창의적인 답을 만들려면 0.9 권장. (Top P와 동시 변경은 권장하지 않음)
2. Maximum length: 최대 토큰 개수
3. Stop sequences: 특정 문자가 생성되면 이후에는 생성을 멈춤
4. Top P: logit 값이 높은 토큰 순서대로 나열 후, 설정한 누적 확률 값에 포함하지 않는 토큰 제거 시 사용하는 기준값. (0.1은 상위 10%의 확률을 가진 토큰으로부터 샘플링)
5. Frequency penalty: 생성한 텍스트에 따라 새로운 토큰에 부과하는 패널티. 동일 텍스트 출력 가능성 감소.
6. Presence penalty: 생성한 텍스트에 따라 새로운 토큰에 부과하는 패널티. 동일 토큰 반복 가능성 감소.
7. Best of: best_of개의 결과를 생성하여 가장 좋은 결과 반환
8. Inject start text: completion 앞에 항상 출력할 텍스트
9. Inject restart text: completion 뒤에항상 출력할 텍스트
10. Show probabilities: 토큰이 생성될 확률을 색상으로 구분하여 표현
    - Off: 꺼짐
    - Most likely: 가장 확률이 높은 것을 색상으로 구분
    - Least likely: 가장 확률이 낮은 것을 색상으로 구분
    - Full Spectrum: 양쪽을 두 가지 색으로 구분

### API

In [99]:
API_KEY = ''
client = OpenAI(api_key=API_KEY)

#### Chat

> ```python
> from openai import OpenAI
> 
> # api_key setting
> API_KEY = ''
> client = OpenAI(api_key=API_KEY)
> 
> # query
> completion = client.chat.completions.create(
>     model="gpt-4o-mini",
>     messages=[
>         {
>             "role": "system",
>             "content": "You are a helpful assistant."
>         },
>         {
>             "role": "assistant",
>             "content": "heeeeeello -> hello"
>         },
>         {
>             "role": "user",
>             "content": "rooooool?"
>         }
>     ],
>     temperature=0.7,
>     max_tokens=64,
> )
> 
> # response
> completion.choices[0].message.content
> ```

<br>

parameters:
- model: 모델 ID
- messages: 모델에 입력할 prompt
    - system role: 모델에 대한 최상위 지침으로, 모델이 무엇을 해야 하는지와 일반적으로 어떻게 행동하고 반응해야 하는지를 설명. <br>
    - assistant message: 이전 요청에 의해 생성된 문장, 또는 few-shot learning을 위한 예제 등 <br>
    - user message: 모델에 특정 유형의 출력을 요청하는 지침 <br>
- max_tokens: 최대 토큰 개수
- temperature: 무작위성 (0~2, default: 1)
- top_p: Nucleus 샘플링 (0~1, default: 1)
- n: 생성 결과 수 (default: 1)
- logprobs: 가능성 높은 토큰에 로그확률 포함
- stop: 토큰 생성을 중지하는 문장 (max: 4)
- frequency_penalty: 동일 텍스트 출력 가능성 감소 (-2~2, default: 0)
- presence_penalty: 동일 토큰 반복 가능성 감소 (-2~2, default: 0)

In [100]:
# user role만 사용용
completion = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {
            'role': 'user',
            'content': '비트코인의 향방에 대해서 너의 생각을 알려줄래?',
        }
    ],
)

In [107]:
print(completion.choices[0].message.content)

비트코인의 향방에 대한 전망은 여러 요인에 따라 달라질 수 있습니다. 다음은 비트코인의 미래에 영향을 미칠 수 있는 몇 가지 주요 요소입니다.

1. **규제 환경**: 각국 정부의 암호화폐에 대한 규제 변화는 비트코인의 가격에 큰 영향을 미칠 수 있습니다. 긍정적인 규제는 시장 신뢰를 높이고, 부정적인 규제는 투자자 심리를 저하시킬 수 있습니다.

2. **시장 수요**: 비트코인이 디지털 자산으로서의 수요가 계속 증가한다면, 가격 상승이 가능할 것입니다. 특히 기관 투자자와 일반 소비자들 사이에서의 수요 변화가 중요합니다.

3. **기술 발전**: 비트코인 관련 기술, 블록체인 기술의 발전과 스케일링 솔루션(예: 라이트닝 네트워크 등)이 비트코인의 사용성을 개선할 경우, 긍정적인 영향을 미칠 수 있습니다.

4. **경제적 요인**: 글로벌 경제 상황, 인플레이션, 금리에 따른 투자자들의 자산 이동도 비트코인의 향방에 큰 영향을 미칩니다. 특히, 비트코인이 '디지털 금'으로 인식되는 경우, 인플레이션 헤지 수단으로서의 수요가 증가할 수 있습니다.

5. **경쟁 암호화폐**: 이더리움, 리플 등 다른 암호화폐와의 경쟁도 중요한 요인입니다. 이들은 각각의 유용성이나 기술적 장점으로 인해 비트코인 시장에 영향을 줄 수 있습니다.

결론적으로, 비트코인의 미래는 복합적인 요인에 의해 결정될 것이며, 기술 및 시장 동향을 주의 깊게 살펴보는 것이 중요합니다. 투자를 고려할 경우 항상 리스크를 감안해야 합니다.


In [108]:
# n값을 조절하여 return 받을 응답 개수 설정
completion = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {
            'role': 'user',
            'content': '비트코인의 향방에 대해서 너의 생각을 알려줄래?',
        }
    ],
    n=3,
)

In [114]:
print(completion.choices[2].message.content)

비트코인의 향방에 대해서는 여러 가지 요인이 영향을 미칠 수 있습니다. 다음은 몇 가지 주요 요소입니다.

1. **규제**: 각국의 정부와 규제 기관이 암호화폐에 대한 규제를 강화하거나 완화하는 방향에 따라 비트코인의 가격과 사용 가능성이 크게 달라질 수 있습니다.

2. **투자자 심리**: 투자자들의 심리는 비트코인 가격에 많은 영향을 미칩니다. 시장의 낙관적 또는 비관적 전망에 따라 가격이 급등하거나 급락할 수 있습니다.

3. **기술적 발전**: 블록체인 기술의 발전과 더불어 비트코인 프로토콜의 업그레이드가 이루어질 경우, 이는 비트코인의 효용성 및 보안을 향상시킬 수 있습니다.

4. **주요 기업과 기관의 수용**: 많은 기업과 기관들이 비트코인을 채택하는 경우, 비트코인의 신뢰성과 안정성이 높아질 수 있습니다. 이는 가격에 긍정적인 영향을 미칠 것입니다.

5. **경제적 환경**: 글로벌 경제 상황, 인플레이션, 통화 정책 등도 비트코인의 수요에 영향을 미칠 수 있습니다. 특히 인플레이션이 높아지면 비트코인을 안전 자산으로 선택하는 경향이 있을 수 있습니다.

결론적으로, 비트코인의 미래는 여러 복합적인 요인에 따라 달라질 수 있으며, 예측하기 어렵습니다. 시장의 변동성이 크기 때문에 신중한 접근이 필요합니다.


In [115]:
# system, user role 설정
completion = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {
            'role': 'system',
            'content': '너는 금융 분야의 전문가야. 금융 현상에 대한 분석 시 긍정적인 이슈와 부정적인 이슈를 나누어 설명해 줘야해.',
        },
        {
            'role': 'user',
            'content': '비트코인의 향방에 대해서 너의 생각을 알려줄래?',
        }
    ],
    n=3,
)

In [121]:
print(completion.choices[2].message.content)

비트코인의 향방에 대해 긍정적인 이슈와 부정적인 이슈로 나눠 설명해드리겠습니다.

### 긍정적인 이슈

1. **제도적 수용 증가**: 많은 국가들이 비트코인을 포함한 암호화폐를 규제하고 법적 지위를 부여하고 있습니다. 이는 비트코인의 신뢰성을 높이고 광범위한 사용을 촉진할 수 있습니다.

2. **디지털 자산으로서의 수요 증가**: 비트코인이 디지털 자산으로 인식되면서 기관 투자자와 기업들이 포트폴리오에 비트코인을 포함하려는 경향이 강해지고 있습니다. 이는 비트코인의 가격 안정성을 향상시킬 수 있습니다.

3. **인플레이션 헤지 수단**: 일부 투자자들은 비트코인이 금과 같은 인플레이션 헤지 수단으로 작용할 수 있다고 보며, 경제 불확실성이 증가함에 따라 비트코인 수요가 높아질 가능성이 있습니다.

4. **기술 혁신**: 블록체인 기술의 발전과 함께 사용자 경험이 개선되고, 거래 속도가 빨라지는 등 비트코인 생태계의 기술적 혁신이 이뤄지고 있습니다. 이러한 발전은 더 많은 사용자를 유입할 수 있는 요인이 될 것입니다.

### 부정적인 이슈

1. **가격 변동성**: 비트코인은 여전히 높은 가격 변동성을 보이고 있습니다. 이는 투자자들에게 불확실성을 제공하고, 일반 사용자들에게는 신뢰성을 떨어뜨릴 수 있습니다.

2. **규제 위험**: 각국 정부의 규제 정책 변화는 비트코인 시장에 부정적인 영향을 미칠 수 있습니다. 특히, 강력한 규제가 도입되면 거래량 감소와 가격 하락을 초래할 가능성이 높습니다.

3. **보안 문제**: 암호화폐 거래소 해킹 사건 등으로 인해 비트코인의 보안 문제가 여전히 우려되고 있습니다. 이러한 사건은 투자자의 신뢰를 떨어뜨릴 수 있으며, 비트코인의 시장 가치에 부정적인 영향을 미칠 수 있습니다.

4. **기술적 한계**: 비트코인은 트랜잭션 처리 속도와 확장성에 한계를 가지고 있어, 대규모 거래를 지원하기에는 부족함이 있습니다. 이는 허용 가능한 거래 비용 상승과 사용자 만족도를 저해할 수 있습니다.

이러한 요소들을 종

In [126]:
# system, assistant, user role 설정. assistant는 example로 볼 수 있음.
completion = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {
            'role': 'system',
            'content': '너는 지시사항에 맞게 결과물을 주는 도우미야.',
        },
        {
            'role': 'assistant',
            'content': 'heeeeeeello -> hello, wooooooooow -> wow',
        },
        {
            'role': 'user',
            'content': 'rooooooooll?',
        }
    ],
)

In [127]:
print(completion.choices[0].message.content)

roll?


In [128]:
prompt = '''
1. 사업의 개요

당사는 본사를 거점으로 한국과 DX 부문 산하 해외 9개 지역총괄 및 DS 부문 산하 해외 5개 지역총괄의 생산ㆍ판매법인, SDC 및 Harman 산하 종속기업 등 232개의 종속기업으로 구성된 글로벌 전자 기업입니다.
사업별로 보면, Set 사업은 DX(Device eXperience) 부문이 TV를 비롯하여 모니터, 냉장고, 세탁기, 에어컨, 스마트폰, 네트워크시스템, 컴퓨터 등을 생산ㆍ판매하며, 부품 사업은 DS(Device Solutions) 부문에서 DRAM, NAND Flash, 모바일AP 등의 제품을 생산ㆍ판매하고,  SDC가 스마트폰용 OLED  패널 등을 생산ㆍ판매하고 있습니다. 또한, Harman에서는 디지털 콕핏(Digital Cockpit), 카오디오 등 전장제품과 포터블/사운드바 스피커 등 소비자오디오 제품 등을 생산ㆍ판매하고 있습니다. 당사는 2021년 12월 조직개편을 통해 기존의 CE 부문과 IM 부문을 DX 부문으로 통합하였으며, 무선사업부의 명칭을 MX(Mobile eXperience) 사업부로 변경하였습니다. 또한 내부 조직체계에 맞추어 SDC를 DS 부문과 별도로 구분하였습니다. ☞ 부문별 사업에 관한 자세한 사항은 '7. 기타 참고사항'의 '다. 사업부문별 현황'과    '라. 사업부문별 요약 재무 현황' 항목을 참고하시기 바랍니다.
지역별로 보면, 국내에서는 DX 부문 및 DS 부문 등을 총괄하는 본사와 35개의 종속기업이 사업을 운영하고 있습니다. 본사는 수원, 구미, 광주, 기흥, 화성, 평택사업장 등으로 구성되어 있으며, 국내 종속기업은 디스플레이 패널을 생산하는 삼성디스플레이㈜와 국내 대리점 판매를 전담하는 삼성전자판매㈜, 제품 수리 서비스를 담당하는 삼성전자서비스㈜, 제품 운송을 담당하는 삼성전자로지텍㈜ 등 비상장 종속기업들로 구성되어 있습니다.

해외(미주, 유럽ㆍCIS, 중동ㆍ아프리카, 아시아 등지)에서는 생산, 판매, 연구활동 등을 담당하는 197개의 비상장 종속기업이 운영되고 있습니다. 미주에는 TV, 스마트폰 등 Set 제품의 미국 판매를 담당하는 SEA(New Jersey, USA), TV 생산을 담당하는 SII(California, USA), 반도체ㆍ디스플레이 패널 판매를 담당하는 SSI(California, USA), 반도체 생산을 담당하는 SAS(Texas, USA), Set 제품 복합 생산법인 SEDA(Brazil), 전장부품사업 등을 담당하는 Harman(Connecticut, USA) 등을 포함하여 총 47개의 판매ㆍ생산 등을 담당하는 법인이 있습니다.
유럽ㆍCIS에는 SEUK(UK), SEG(Germany), SEF(France), SEI(Italy), SERC(Russia)등 Set 제품 판매법인과 SEH(Hungary), SERK(Russia) 등 TV 생산법인, 가전제품 생산법인 SEPM(Poland) 등을 포함하여 총 68개의 법인이 운영되고 있습니다. 중동ㆍ아프리카에는 SGE(UAE), SSA(South Africa) 등 Set 제품 판매법인과 SEEG (Egypt), SSAP(South Africa) 등 TV 생산법인을 포함한 총 20개 법인이 운영되고 있습니다. 아시아(중국 제외)에는 SESP(Singapore), SEAU(Australia), SEPCO(Philippines),  SME(Malaysia) 등 판매법인과 스마트폰 등 생산법인 SEVㆍSEVT(Vietnam), TV 등 생산법인 SEHC(Vietnam), 디스플레이 패널 생산법인 SDV(Vietnam), 복합 생산법인 SIEL(India) 등을 포함한 총 32개의 법인이 운영되고 있습니다. 중국에는 SCIC(Beijing), SEHK(Hong Kong) 등 Set 제품 판매법인과 SSS(Shanghai), SSCX(Xian) 등 반도체ㆍ디스플레이 패널 판매법인, SSEC(Suzhou) 등 Set 제품 생산법인, SCS(Xian) 등 반도체 생산법인을 포함하여 총 30개의 법인이 운영되고 있습니다.
2023년 당사의 매출은 258조 9,355억원으로 전년 대비 14.3% 감소하였으며, 주요 매출처로는 Apple, Best Buy, Deutsche Telekom, Qualcomm, Verizon 등(알파벳순)이 있습니다.

## 재무제표의 사업의 개요 중 해외 지역에서 어떠한 사업을 영위하고 있는지 파악하려고 해.
예를 들면 미주 지역에서는 TV, 스마트폰, 반도체, 디스플레이 패널 등의 사업을 영위하고 있어.
이 예시를 참고하여 다른 해외 지역에서는 어떠한 사업을 영위하고 있는지 알려줄래?
'''

completion = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {
            'role': 'system',
            'content': '너는 재무제표를 받아 정리해주는 봇이야.',
        },
        {
            'role': 'user',
            'content': prompt,
        }
    ],
)

In [129]:
print(completion.choices[0].message.content)

해외 지역별 사업 영위 현황은 다음과 같습니다:

1. **미주 지역**:
   - **사업 내용**: TV, 스마트폰 등 Set 제품의 판매
   - **생산 및 판매 법인**: 
     - SEA (New Jersey, USA): TV, 스마트폰 등 Set 제품 판매
     - SII (California, USA): TV 생산
     - SSI (California, USA): 반도체 및 디스플레이 패널 판매
     - SAS (Texas, USA): 반도체 생산
     - SEDA (Brazil): Set 제품 복합 생산
     - Harman (Connecticut, USA): 전장부품사업 및 오디오 제품 판매

2. **유럽ㆍCIS 지역**:
   - **사업 내용**: Set 제품 판매 및 생산
   - **생산 및 판매 법인**:
     - SEUK (UK), SEG (Germany), SEF (France), SEI (Italy), SERC (Russia): Set 제품 판매
     - SEH (Hungary), SERK (Russia): TV 생산
     - SEPM (Poland): 가전제품 생산

3. **중동ㆍ아프리카 지역**:
   - **사업 내용**: Set 제품 판매 및 TV 생산
   - **생산 및 판매 법인**:
     - SGE (UAE), SSA (South Africa): Set 제품 판매
     - SEEG (Egypt), SSAP (South Africa): TV 생산

4. **아시아 (중국 제외) 지역**:
   - **사업 내용**: 판매 및 다양한 제품의 생산
   - **생산 및 판매 법인**:
     - SESP (Singapore), SEAU (Australia), SEPCO (Philippines), SME (Malaysia): 판매법인
     - SEV, SEVT (Vietnam): 스마트폰 등 생산법인
     - SEHC (Vietnam): TV 생산법인
     - SDV

#### Structured Outputs

지정된 형식으로 값을 return. <br>
pytantic의 BaseModel을 상속받아 속성 지정. <br>

```python
class Step(BaseModel):
    explanation: str
    output: str

class Reasoning(BaseModel):
    steps: list[Step]
    final_answer: str

completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": role
        },
        {
            "role": "user",
            "content": prompt,
        }
    ],
    response_format=Reasoning,
)
```

In [130]:
class Step(BaseModel):
    explanaton: str
    output: str

class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

completion = client.beta.chat.completions.parse(
    model='gpt-4o-mini',
    messages=[
        {
            'role': 'system',
            'content': '너는 수학을 도와주는 튜터야. 스텝 바이 스텝으로 유저에게 풀이를 제시해줘.'
        },
        {
            'role': 'user',
            'content': '8x + 7 = -23에서 x값을 어떻게 구할 수 있어?'
        }
    ],
    response_format=MathReasoning,
)

In [146]:
eval(completion.choices[0].message.content)

{'steps': [{'explanaton': '먼저, 방정식 8x + 7 = -23에서 7을 양쪽에서 빼줍니다.',
   'output': '8x + 7 - 7 = -23 - 7'},
  {'explanaton': '이제 방정식이 8x = -30으로 간단해졌습니다.', 'output': '8x = -30'},
  {'explanaton': '다음으로, x를 구하기 위해 양쪽을 8로 나눕니다.', 'output': 'x = -30 / 8'},
  {'explanaton': '이제 -30 / 8을 간단히 하면 -15 / 4가 됩니다.', 'output': 'x = -15 / 4'}],
 'final_answer': 'x = -15/4'}

In [154]:
prompt = '''
1. 사업의 개요

당사는 본사를 거점으로 한국과 DX 부문 산하 해외 9개 지역총괄 및 DS 부문 산하 해외 5개 지역총괄의 생산ㆍ판매법인, SDC 및 Harman 산하 종속기업 등 232개의 종속기업으로 구성된 글로벌 전자 기업입니다.
사업별로 보면, Set 사업은 DX(Device eXperience) 부문이 TV를 비롯하여 모니터, 냉장고, 세탁기, 에어컨, 스마트폰, 네트워크시스템, 컴퓨터 등을 생산ㆍ판매하며, 부품 사업은 DS(Device Solutions) 부문에서 DRAM, NAND Flash, 모바일AP 등의 제품을 생산ㆍ판매하고,  SDC가 스마트폰용 OLED  패널 등을 생산ㆍ판매하고 있습니다. 또한, Harman에서는 디지털 콕핏(Digital Cockpit), 카오디오 등 전장제품과 포터블/사운드바 스피커 등 소비자오디오 제품 등을 생산ㆍ판매하고 있습니다. 당사는 2021년 12월 조직개편을 통해 기존의 CE 부문과 IM 부문을 DX 부문으로 통합하였으며, 무선사업부의 명칭을 MX(Mobile eXperience) 사업부로 변경하였습니다. 또한 내부 조직체계에 맞추어 SDC를 DS 부문과 별도로 구분하였습니다. ☞ 부문별 사업에 관한 자세한 사항은 '7. 기타 참고사항'의 '다. 사업부문별 현황'과    '라. 사업부문별 요약 재무 현황' 항목을 참고하시기 바랍니다.
지역별로 보면, 국내에서는 DX 부문 및 DS 부문 등을 총괄하는 본사와 35개의 종속기업이 사업을 운영하고 있습니다. 본사는 수원, 구미, 광주, 기흥, 화성, 평택사업장 등으로 구성되어 있으며, 국내 종속기업은 디스플레이 패널을 생산하는 삼성디스플레이㈜와 국내 대리점 판매를 전담하는 삼성전자판매㈜, 제품 수리 서비스를 담당하는 삼성전자서비스㈜, 제품 운송을 담당하는 삼성전자로지텍㈜ 등 비상장 종속기업들로 구성되어 있습니다.

해외(미주, 유럽ㆍCIS, 중동ㆍ아프리카, 아시아 등지)에서는 생산, 판매, 연구활동 등을 담당하는 197개의 비상장 종속기업이 운영되고 있습니다. 미주에는 TV, 스마트폰 등 Set 제품의 미국 판매를 담당하는 SEA(New Jersey, USA), TV 생산을 담당하는 SII(California, USA), 반도체ㆍ디스플레이 패널 판매를 담당하는 SSI(California, USA), 반도체 생산을 담당하는 SAS(Texas, USA), Set 제품 복합 생산법인 SEDA(Brazil), 전장부품사업 등을 담당하는 Harman(Connecticut, USA) 등을 포함하여 총 47개의 판매ㆍ생산 등을 담당하는 법인이 있습니다.
유럽ㆍCIS에는 SEUK(UK), SEG(Germany), SEF(France), SEI(Italy), SERC(Russia)등 Set 제품 판매법인과 SEH(Hungary), SERK(Russia) 등 TV 생산법인, 가전제품 생산법인 SEPM(Poland) 등을 포함하여 총 68개의 법인이 운영되고 있습니다. 중동ㆍ아프리카에는 SGE(UAE), SSA(South Africa) 등 Set 제품 판매법인과 SEEG (Egypt), SSAP(South Africa) 등 TV 생산법인을 포함한 총 20개 법인이 운영되고 있습니다. 아시아(중국 제외)에는 SESP(Singapore), SEAU(Australia), SEPCO(Philippines),  SME(Malaysia) 등 판매법인과 스마트폰 등 생산법인 SEVㆍSEVT(Vietnam), TV 등 생산법인 SEHC(Vietnam), 디스플레이 패널 생산법인 SDV(Vietnam), 복합 생산법인 SIEL(India) 등을 포함한 총 32개의 법인이 운영되고 있습니다. 중국에는 SCIC(Beijing), SEHK(Hong Kong) 등 Set 제품 판매법인과 SSS(Shanghai), SSCX(Xian) 등 반도체ㆍ디스플레이 패널 판매법인, SSEC(Suzhou) 등 Set 제품 생산법인, SCS(Xian) 등 반도체 생산법인을 포함하여 총 30개의 법인이 운영되고 있습니다.
2023년 당사의 매출은 258조 9,355억원으로 전년 대비 14.3% 감소하였으며, 주요 매출처로는 Apple, Best Buy, Deutsche Telekom, Qualcomm, Verizon 등(알파벳순)이 있습니다.

## 재무제표의 사업의 개요 중 해외 지역에서 어떠한 사업을 영위하고 있는지 파악하려고 해.
예를 들면 미주 지역에서는 TV, 스마트폰, 반도체, 디스플레이 패널 등의 사업을 영위하고 있어.
이 예시를 참고하여 다른 해외 지역에서는 어떠한 사업을 영위하고 있는지 알려줄래?
'''

class Detail(BaseModel):
    region: str
    business_description: str
    manufacturing_and_sales_corporation: str

class Summarize(BaseModel):
    result: list[Detail]

completion = client.beta.chat.completions.parse(
    model='gpt-4o-mini',
    messages=[
        {
            'role': 'system',
            'content': '너는 재무제표를 받아 정리해주는 봇이야.',
        },
        {
            'role': 'user',
            'content': prompt,
        }
    ],
    response_format=Summarize
)

In [155]:
eval(completion.choices[0].message.content)

{'result': [{'region': '미주',
   'business_description': 'TV, 스마트폰, 반도체, 디스플레이 패널의 판매 및 생산업무 담당',
   'manufacturing_and_sales_corporation': 'SEA, SII, SSI, SAS, SEDA, Harman'},
  {'region': '유럽ㆍCIS',
   'business_description': 'Set 제품의 판매 및 TV, 가전제품 생산업무 담당',
   'manufacturing_and_sales_corporation': 'SEUK, SEG, SEF, SEI, SERC, SEH, SERK, SEPM'},
  {'region': '중동ㆍ아프리카',
   'business_description': 'Set 제품의 판매 및 TV 생산업무 담당',
   'manufacturing_and_sales_corporation': 'SGE, SSA, SEEG, SSAP'},
  {'region': '아시아(중국 제외)',
   'business_description': '판매 및 스마트폰, TV, 디스플레이 패널의 생산업무 담당',
   'manufacturing_and_sales_corporation': 'SESP, SEAU, SEPCO, SME, SEV, SEVT, SEHC, SDV, SIEL'},
  {'region': '중국',
   'business_description': 'Set 제품 판매 및 반도체, 디스플레이 패널의 판매 및 생산업무 담당',
   'manufacturing_and_sales_corporation': 'SCIC, SEHK, SSS, SSCX, SSEC, SCS'}]}

#### Embedding

embedding: https://platform.openai.com/docs/api-reference/embeddings

> ```python
> from openai import OpenAI
> 
> # api_key setting
> API_KEY = ''
> client = OpenAI(api_key=API_KEY)
> 
> # query
> completion = client.embeddings.create(
>     model="text-embedding-3-small",
>     input='',
>     encoding_format="float",
> )
> 
> # response
> completion.choices[0].message.content
> ```

<br>

parameters:
- model: 모델 ID
- input: 모델에 입력할 text
- encoding_format: 부동소숫점 인코딩

In [157]:
embedding = client.embeddings.create(
    model='text-embedding-3-small',
    input='hello world',
)

In [165]:
# embedding된 vector의 차원
len(embedding.data[0].embedding)

1536

##### Practice

###### 유튜브 댓글 clustering


1. data: comments_minheejin.pickle (day41) ~15:10
2. 텍스트 임베딩
3. 임베딩된 결과를 가지고 cosine similarity 계산
4. simliarity가 높은 값을 선택하여 실제로 댓글이 유사한지 판단

In [ ]:
data = pd.read_pickle('./data/comments_minheejin.pickle').iloc[:100]

def embedding(comment: str, model: str = 'text-embedding-3-small'):
    try:
        embedding = client.embeddings.create(
            model=model,
            input=comment,
        )
        embedding = embedding.data[0].embedding
    except:
        embedding = -1

    return embedding

data['vector'] = data.textOriginal.apply(lambda x: embedding(x))

In [184]:
vector = data.vector.tolist()
similarity = cosine_similarity(vector)
np.fill_diagonal(similarity, 0)
np.where(similarity > 0.6)

(array([16, 49, 49, 60, 67, 70, 72, 88, 91, 95], dtype=int64),
 array([49, 16, 91, 67, 60, 72, 70, 95, 49, 88], dtype=int64))

###### 회사 clustering

1. data: crawl_data.pickle (day40)
2. 텍스트 임베딩 (회사의 개요)
3. 임베딩된 결과를 가지고 cosine similarity 계산
4. simliarity가 높은 값을 선택하여 실제로 댓글이 유사한지 판단

In [207]:
data = pd.read_pickle('./data/crawl_data.pickle').query('dates=="2022.03"')
data = data.query('corp_name.str.contains("은행|보험|카드|저축|캐피탈|기업|국민|신한|농협|우리|생명")')

data['processed_overview'] = (data['회사의 개요']
                              .apply(lambda x: BeautifulSoup(x, 'lxml').text)
                              .str.replace('\s+', ' ', regex=True)
                             )

data['vector'] = data.processed_overview.apply(lambda x: embedding(x))
data = data.loc[data.vector.apply(type) != int]

In [244]:
vector = data.vector.tolist()
similarity = cosine_similarity(vector)
np.fill_diagonal(similarity, 0)
np.where(similarity > 0.8)

(array([ 0,  0,  0,  0,  2,  3,  3,  3,  3,  3,  4,  5,  6,  6,  6,  7,  7,
         7,  7,  9,  9,  9,  9,  9, 10, 12, 12, 12, 12, 12, 14, 16],
       dtype=int64),
 array([ 3,  7,  9, 12,  5,  0,  6,  7,  9, 12, 10,  2,  3,  9, 12,  0,  3,
         9, 12,  0,  3,  6,  7, 12,  4,  0,  3,  6,  7,  9, 16, 14],
       dtype=int64))

In [245]:
data.iloc[[0, 7]]

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm,modified_report_nm,dates,회사의 개요,사업의 개요,processed_overview,vector
30,00113058,한화생명,088350,Y,분기보고서 (2022.03),20220516002568,한화생명,20220516,,분기보고서 (2022.03),2022.03,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...","<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",1. 회사의 개요 가. 연결대상 종속회사 개황(연결재무제표를 작성하는 주권상장법인...,"[0.023187456652522087, 0.03689926862716675, 0...."
148,00126292,삼성카드,029780,Y,분기보고서 (2022.03),20220516002162,삼성카드,20220516,,분기보고서 (2022.03),2022.03,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...","<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",1. 회사의 개요 가. 연결대상 종속회사 개황- 당사 종속기업은 다음과 같습니다....,"[0.005950961261987686, 0.05119762569665909, 0...."


###### prompt engineering을 통한 회사 클러스터링

1. prompting을 통해 회사의 개요 전처리
2. 처리된 회사의 개요 embedding
3. embedding된 결과 cosine similarity 계산
4. threshold를 조절하며 유사한 두 회사를 확인

In [ ]:
data = pd.read_pickle('./data/crawl_data.pickle').query('dates=="2022.03"')
data = data.query('corp_name.str.contains("은행|보험|카드|저축|캐피탈|기업|국민|신한|농협|우리|생명")')

data['processed_overview'] = (data['회사의 개요']
                              .apply(lambda x: BeautifulSoup(x, 'lxml').text)
                              .str.replace('\s+', ' ', regex=True)
                             )

### Prompt Engineering

언어 모델을 효과적으로 활용하기 위해 입력 프롬프트를 설계하고 최적화하는 과정. <br>
프롬프트의 구조, 언어, 길이 등을 조정하여 모델이 더 정확하고 유용한 응답을 생성할 수 있도록 특정한 형식이나 내용을 가진 질문이나 지시문을 작성. <br>

#### 명확한 prompt 작성

##### 쿼리에 세부 정보를 포함

query에 중요한 세부사항이나 context 제공

<br>

| Worse                                         | Better                                                                                                       |
|-----------------------------------------------|--------------------------------------------------------------------------------------------------------------|
| Excel에서 숫자를 어떻게 더해?              | Excel에서 달러 금액의 한 행을 자동으로 더하고, 모든 총합이 "총계"라는 열에 오른쪽에 표시되도록 하려면 어떻게 해야해?       |
| 누가 대통령이야?                           | 2021년 멕시코의 대통령은 누구였으며, 선거는 얼마나 자주 열려?                                          |
| 피보나치 수열을 계산하는 코드를 작성해줘.   | TypeScript 함수를 작성하여 피보나치 수열을 효율적으로 계산해. 각 코드 조각이 무엇을 하는지와 왜 그렇게 작성되었는지 설명하는 주석을 충분히 달아줘. |
| 회의 노트를 요약해줘.                       | 회의 노트를 한 문단으로 요약한 후, 발언자 목록과 각 발언자의 주요 요점을 markdown 리스트 형식으로 작성해줘. 마지막으로, 발언자들이 제안한 다음 단계 또는 실행 항목을 나열해줘. |

##### 모델에게 특정 역할을 맡기기

시스템 메시지를 사용하여 모델이 응답에서 사용할 역할을 지정 <br>

<br>

> ```python
> [
>     {
>         "role": "system",
>         "content": "도움이 필요한 내용을 작성할 때마다, 각 단락에 최소한 하나의 농담이나 장난스러운 댓글을 포함하여 응답."
>     },
>     {
>         "role": "user",
>         "content": "짧은 통보로 제때 납품해 준 스틸 볼트 공급업체에 감사의 메시지를 작성해줘. 덕분에 중요한 주문을 전달할 수 있었습니다라고."
>     }
> ]
> ```

In [251]:
completion = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {
            'role': 'system',
            'content': '''
            너는 유저의 질문에 답변하는 봇이야.
            유저가 너를 찾을 때는 일반적으로 화가 많이 나있을 거니 일단 죄송하다는 이야기를 하자.
            그리고 분위기 전환을 위해서 위트가 넘치는 표현으로 고객의 화를 풀어주는 게 좋을 거야.
            '''
        },
        {
            'role': 'user',
            'content': '''
            우일이형 광고를 이렇게 받아먹었어? 아니 이거는 아니지. 구독자를 기만하는 건 말이 안 되잖아.
            구독 취소할게.
            '''
        }
    ]
)

In [252]:
print(completion.choices[0].message.content)

죄송합니다! 화가 나신 것 같아 마음이 아프네요. 기분이 이렇게 상하셨다니, 저도 마음이 무겁습니다. 하지만 구독 취소는 섭섭하네요. 저희가 여러분을 기쁘게 해드릴 수 있도록 변신할 수는 없을까요? 마치 연두부가 변신하는 것처럼! 한 번만 이야기해주시면 더 나은 방향으로 나아갈 방법을 찾도록 하겠습니다!


##### 구분자 사용

텍스트의 섹션을 다르게 처리할 수 있도록 명확하게 구분하는 데 도움이 됨. <br>
&nbsp;&nbsp;&nbsp;&nbsp; ex)""", XML 태그, 섹션 제목 등

<br>

> ```python
> [
>     {
>         "role": "user",
>         "content": "세 개의 따옴표로 구분된 텍스트를 요약해줘. \n '''{text}'''"
>     },
> ]
> [
>     {
>         "role": "system",
>         "content": "XML 태그로 구분된 두 개의 기사가 동일한 주제에 대한 주장을 제공할 거야. 각 기사의 주장을 요약한 후, 어느 기사가 더 나은 주장을 하는지 설명해줘."
>     },
>     {
>         "role": "user",
>         "content": f"<article> {article1} </article> \n\n <article> {article2} </article>"
>     },
> ]
> [
>     {
>         "role": "system",
>         "content": "paper의 abstract와 title을 제공받을 거야. title은 paper의 주제를 잘 전달하면서도 눈길을 끌어야 해. 제목이 이러한 기준을 충족하지 않으면, 5개의 대안을 제안해줘."
>     },
>     {
>         "role": "user",
>         "content": "abstract: {abstract} \n\n title: {title}"
>     },
> ]
> ```

In [253]:
text1 = '''
북한이 동해선 육로를 폭파했다고 보도하면서 내놓은 사진이 우리 군 합동참모본부가 촬영한 영상을 ‘불펌’(불법 퍼가기)한 것으로 보인다는 분석이 나왔다.

17일 북한 조선중앙통신은 지난 15일 감행한 동해선과 경의선 육로 폭파 소식을 이날 폭파 장면 사진과 함께 보도했다.

북한은 지난 15일 남북을 연결하는 경의선과 동해선 도로 일부를 폭파했다. 북한이 남북 단절과 연결로 요새화를 공식 선언한 지 엿새 만이다. 이에 우리 군은 군사분계선(MDL) 이남 지역으로 대응 사격을 실시했다.

북한이 공개한 사진은 총 3장인데 그 가운데 동해선 폭파 사진 1장은 폭파 당시 합참이 감시장비로 촬영한 영상에 포함된 장면과 거의 동일하다.

합참 영상에서는 북한 사진상 우측에 나타난 파란 표지판과 흰색 가로등, 연기가 퍼지는 모양, 하단의 우거진 수풀이 같은 모습으로 잡힌 장면을 찾을 수 있다.

북한 사진의 색깔이 조금 더 흐릿한 편이고 연기 모양이 조금 다르기는 하나 이는 보정 작업 과정에서 나타난 차이일 수 있다.

한 사진 전문가는 “연기 등이 미세하게 달라 보이지만, 보정에 따른 경계선 차이로 보인다”며 “같은 사진으로 추정된다”고 분석했다.

이와 관련해 군 관계자는 “북한이 우리 군과 동일 위치에서 촬영하는 것은 불가능하고, 동일 각도에서 촬영했을 수는 있다”고 설명했다.

그러나 동일 각도라도 촬영 고도까지 일치하기는 어렵고, 이 정도의 유사도가 나오기는 쉽지 않기 때문에 북한이 합참 촬영본을 ‘불펌’한 것이라는 분석이 나왔다.

앞서 북한은 지난 15일 폭파 당시 경의선 현장에 촬영 인원을 파견한 모습이 우리 군 감시장비에 포착되기도 했다. 하지만 동해선 폭파 현장에서는 북한 인원이 촬영하는 모습이 식별되지 않았다.

북한은 폭파 감행 이튿날인 지난 16일에는 폭파와 관련해 아무런 소식을 내놓지 않았다. 이에 현장 사진을 제대로 챙기지 못한 까닭에 이를 수습하느라 하루를 보낸 뒤 부랴부랴 남측 동영상을 가져다 쓴 것 아니냐는 추측이 제기된다.
'''

text2 = '''
[서울=뉴시스]이지용 기자 = LG전자가 한국IR협의회가 주관하는 '2024 한국IR대상'에서 기업부문 최고상인 '금융위원장상'을 수상했다고 17일 밝혔다.

LG전자는 이날 서울 영등포구 한국거래소 서울사무소에서 열린 2024 한국IR대상 행사에서 대상(금융위원장상)을 수상했다.

한국IR대상은 한국거래소 산하 한국IR협의회가 지난 2001년부터 매년 효과적인 IR활동을 통해 자본시장의 건전한 발전에 기여한 기업 및 개인에게 시상한다.

LG전자는 가전 등 주력사업에서 미래 지향적 사업구조로 변화하며 지속가능한 성장을 추진하겠다는 중·장기 전략방향 '2030 미래비전'을 투자자 소통에 반영한 점을 높게 평가 받았다. 또 진정성 있는 기업가치 및 주주가치 제고 행보를 이어갔다는 평이다.

LG전자는 지난해 7월 '2030 미래비전' 발표를 시작으로 주주총회, 해외 투자자 대상 기업설명회, 인베스터 포럼 등 주요 IR 행사마다 최고경영자(CEO), 최고재무책임자(CFO) 등 경영진이 직접 나서 소통을 이어가고 있다. 정기 실적발표 콘퍼런스콜도 올해부터 CFO 주관으로 격상해 진행 중이다.

김창태 LG전자 CFO는 "앞으로도 기업활동 전반에 걸쳐 진정성 있는 소통을 지속하며 투자자 신뢰를 높여 나갈 것"이라고 말했다.
'''

In [255]:
completion = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {
            'role': 'system',
            'content': f'''세 개의 따옴표로 감싸진 텍스트의 내용만 요약하는 봇이야. 작은 따옴표(') 또는
            큰 따옴표(") 세 개로 감싸져 있는 텍스트의 내용만을 요약할 거야'''
        },
        {
            'role': 'user',
            'content': f"세 개의 따옴표로 감싸진 텍스트의 내용만 요약해줘. \n {text1} \n '''{text2}'''"
        }
    ]
)
completion.choices[0].message.content

"'''\nLG전자가 '2024 한국IR대상'에서 기업부문 최고상인 '금융위원장상'을 수상하였고, 이는 효과적인 IR활동을 통해 자본시장 발전에 기여한 결과이다. LG전자는 지속가능한 성장을 위한 중·장기 전략 '2030 미래비전'을 투자자 소통에 반영하여 높은 평가를 받았다. 경영진이 직접 주요 IR 행사에 참여하고, 정기 실적발표를 CFO 주관으로 진행하는 등 진정성 있는 소통을 강조하고 있다.\n'''"

##### 세부 step으로 분할

일부 작업은 단계를 명확하게 지정하는 것이 가장 좋은 결과를 보임. <br>
단계를 명시적으로 작성하면 모델이 이를 따라가기가 더 쉬워질 수 있음. <br>

<br>

> ```python
> system_prompt = '''
> "다음 단계별 지침을 사용하여 사용자 입력에 응답."
> 
> 1단계 - 사용자가 세 개의 따옴표로 구분된 텍스트를 제공할 거야. 이 텍스트를 한 문장으로 요약하고 'Summary: '라는 접두사를 붙여줘.
> 2단계 - 1단계의 요약을 스페인어로 번역하고 'Translation: '라는 접두사를 붙여줘."
> '''
> 
> 
> [
>     {
>         "role": "system",
>         "content": system_prompt,
>     },
>     {
>         "role": "user",
>         "content": """{text}""",
>     }
> ]
> ```


In [256]:
system_prompt = '''
다음의 단계 지침을 사용하여 주어진 데이터를 처리할 수 있는 코드 작성.

1단계: 사용자가 dataset을 전달할 거야. 이는 머신러닝의 학습을 위한 데이터이며
supervised task를 진행할 수 있는 label이 포함된 데이터가 제공될 거야.

2단계: 1단계의 입력에서 결측치와 중복값을 확인하여, 중복이나 결측 발생 시
제거.

3단계: label encoder라고 하는 데이터 프레임과 범주형 변수로 변환할 컬럼 이름이
담긴 리스트를 입력으로 받는 함수를 사용하여 데이터를 범주형 변수로 변환.

4단계: train_test_split 함수를 사용하여 3단계에서 처리된 데이터를
train과 test data로 분할할 거야.

5단계: label data를 예측하는 모델 학습.

6단계: 학습한 모델을 통해 얻어진 test 데이터의 예측 값과 실제 값을
비교하는 시각화 코드 작성.
'''

completion = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {
            'role': 'system',
            'content': system_prompt
        },
        {
            'role': 'user',
            'content': 'sklearn의 diamond dataset'
        }
    ]
)

print(completion.choices[0].message.content)

파이썬을 사용하여 주어진 단계에 따라 sklearn의 diamond 데이터셋을 처리하는 코드 예제는 다음과 같습니다. 우선 필요한 라이브러리를 설치하고 데이터를 불러오는 것에서 시작합니다.

```python
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
import numpy as np

# 1단계: 다이아몬드 데이터셋 로드
from sklearn.datasets import fetch_openml

# Load the diamond dataset
diamonds = fetch_openml('Diamonds', version=1, as_frame=True)
df = diamonds.frame

# 2단계: 결측치 및 중복값 처리
# 결측치 제거
df.dropna(inplace=True)

# 중복값 제거
df.drop_duplicates(inplace=True)

# 3단계: label encoder 함수 정의
def encode_categorical(df, categorical_cols):
    le = LabelEncoder()
    for col in categorical_cols:
        df[col] = le.fit_transform(df[col])
    return df

# 범주형 변수 지정
categorical_columns = ['cut', 'color', 'clarity']
df_encoded = encode_categorical(df, categorical_column

##### example 제공

예를 제공하여 모든 생성 가능한 조합에서 답을 찾는 것보다 효과적으로 답을 찾을 수 있음. <br>
이를 **few-shot prompt**라고 함. <br>

> ```python
> [
>     {
>         "role": "system",
>         "content": "일관된 스타일로 응답."
>     },
>     {
>         "role": "user",
>         "content": "인내에 대해 가르쳐 줄래?"
>     },
>     {
>         "role": "assistant",
>         "content": "깊은 계곡을 파는 강은 소박한 샘에서 흐르며, 가장 웅장한 교향곡은 한 음에서 시작되고, 가장 복잡한 태피스트리는 고독한 실로부터 시작."
>     },
>     {
>         "role": "user",
>         "content": "바다에 대해 가르쳐 줄래?"
>     }
> ]
> ```

In [265]:
completion = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {
            'role': 'system',
            'content': '일관된 스타일로 응답.',
        },
        {
            'role': 'user',
            'content': '한국하면 떠오르는 단어를 나열해 줄래?',
        },
        {
            'role': 'assistant',
            'content': '1. 한강, 2. BTS, 3. 기생충'
        },
        {
            'role': 'user',
            'content': '미국하면 떠오르는 단어를 나열해 줄래?'
        },
    ],
)

print(completion.choices[0].message.content)

1. 자유, 2. 뉴욕, 3. 할리우드, 4. 자동차, 5. 미국 독립전쟁


##### 길이 명시

모델에게 특정한 목표 길이의 출력을 생성하도록 요청할 수 있음. <br>
길이는 단어, 문장, 단락, 글머리 기호 등의 개수로 지정 가능. <br>
&nbsp;&nbsp;&nbsp;&nbsp;모델에게 특정한 단어 수를 생성하도록 지시하는 것은 정확하게 작동하지 않을 수 있음.

<br>

> ```python
> [
>     {
>         "role": "user",
>         "content": "세 개의 따옴표로 구분된 텍스트를 약 50단어로 요약해줄래?. \n{text}"
>     },
> ]
> [
>     {
>         "role": "user",
>         "content": "세 개의 따옴표로 구분된 텍스트를 2단락으로 요약하세요. \n{text}"
>     },
> ]
> [
>     {
>         "role": "user",
>         "content": "세 개의 따옴표로 구분된 텍스트를 3개의 핵심 포인트로 요약하세요. \n{text}"
>     },
> ]
> ```

#### reference 제공

##### reference를 사용하여 응답하도록 지시

신뢰할 수 있는 정보를 모델에 제공하고 이 정보가 현재 쿼리와 관련이 있다면, 해당 정보를 사용하여 답변을 작성하도록 모델에게 지시 가능.

<br>

> ```python
> [
>     {
>         "role": "system",
>         "content": "제공된 세 개의 따옴표로 구분된 기사를 사용하여 질문에 답하세요. 기사가 아닌 경우 '답을 찾을 수 없습니다.'라고 작성하세요."
>     },
>     {
>         "role": "user",
>         "content": """{text}"""
>     },
>     {
>         "role": "user",
>         "content": "Question: {question}"
>     }
> ]
> ```

##### reference로부터 인용하여 답변하도록 지시

입력이 관련 지식으로 보완된 경우, 모델에게 제공된 문서의 특정 구문을 참조하여 답변에 인용을 추가하도록 요청하는 것은 간단. <br>
출력에 포함된 인용은 제공된 문서 내에서 문자열 일치를 통해 프로그래밍적으로 검증 가능 <br>

<br>

> ```python
> prompt = '''
> """{article}"""
> 
> Question: {question}
> '''
> 
> [
>     {
>         "role": "system",
>         "content": "제공된 세 개의 따옴표로 구분된 기사를 사용하여 질문에 답하여라. 기사가 아닌 경우 '답을 찾을 수 없습니다.'라고 작성."
>     },
>     {
>         "role": "user",
>         "content": prompt,
>     }
> ]
> ```

#### 복잡한 task를 simple한 task로 분리

##### 가장 관련성 높은 지침을 식별하기 위해 의도 분류 사용

많은 독립적인 지침 세트를 필요로 하는 작업에서는 먼저 문의 유형을 분류하고 이 분류를 사용하여 필요한 지침을 결정하는 것이 유용할 수 있음. <br>
이를 위해 고정된 범주를 정의하고 특정 범주에서 작업을 처리하는 데 관련된 지침을 하드코딩. <br>
이 과정은 작업을 일련의 단계로 분해하는 데도 재귀적으로 적용 가능. <br>

이 접근 방식의 장점은 각 문의에 필요한 지침만 포함되어 있어 작업의 다음 단계를 수행하는 데 필요한 정보를 제공함으로써 오류율을 낮출 수 있다는 것. <br>
더 큰 프롬프트는 실행 비용이 더 높기 때문에 비용 절감 효과도 기대 가능. <br>

> ```python
> system_prompt = '''
> 고객 서비스 문의가 제공될 것. 각 문의를 기본 범주와 세부 범주로 분류. JSON 형식으로 출력하고, 키는 primary와 secondary로 분류.
> 
> 기본 범주: 청구, 기술 지원, 계정 관리 또는 일반 문의.
> 
> 청구 세부 범주:
> - 구독 해지 또는 업그레이드
> - 결제 방법 추가
> - 청구 설명
> - 요금 이의 제기
> - 기술 지원 세부 범주:
> 
> 문제 해결
> - 장치 호환성
> - 소프트웨어 업데이트
> 
> 계정 관리 세부 범주:
> - 비밀번호 재설정
> - 개인 정보 업데이트
> - 계정 해지
> - 계정 보안
> 
> 일반 문의 세부 범주:
> - 제품 정보
> - 가격
> - 피드백
> - 사람과 대화하기
> '''
>
> [
>     {
>         "role": "system",
>         "content": system_prompt,
>     },
>     {
>         "role": "user",
>         "content": "인터넷이 다시 작동하도록 해야 하는데 어떻게 해야할까?",
>     }
> ]
> ```

<br>

고객 문의 분류를 기반으로 모델이 다음 단계를 처리할 수 있도록 보다 구체적인 지침을 제공 가능. <br>
&nbsp;&nbsp;&nbsp;&nbsp; ex) 고객이 "문제 해결"에 대한 도움이 필요하다고 가정.

> ```python
> system_prompt = f'''
> 고객 서비스 문의가 제공될 건데, 기술 지원 맥락에서 문제 해결이 필요한 경우 사용자를 도와줄래?:
> 
> - 라우터에 연결된 모든 케이블이 제대로 연결되어 있는지 확인하도록 요청. 시간이 지남에 따라 케이블이 느슨해지는 경우가 흔함.
> - 모든 케이블이 연결되어 있는데도 문제가 지속된다면, 사용하고 있는 라우터 모델이 무엇인지 물어봐.
> - 이제 장치를 재시작하는 방법을 안내:
> -- 모델 번호가 MTD-327J인 경우, 빨간 버튼을 누르고 5초 동안 유지한 후, 연결 상태를 테스트하기 전에 5분 기다리라고 안내.
> -- 모델 번호가 MTD-327S인 경우, 전원 플러그를 뽑았다가 다시 꽂고, 연결 상태를 테스트하기 전에 5분 기다리라고 안내.
> - 장치를 재시작하고 5분을 기다린 후에도 문제가 지속된다면, IT 지원팀에 연결하라는 메시지를 출력: "IT support requested".
> - 사용자가 이 주제와 관련 없는 질문을 시작하면, 문제 해결에 대한 현재 채팅을 종료하고 싶어하는지 확인한 후, 요청을 다음 분류 체계에 따라 분류.
> 
> {<primary/secondary classification theme>}
> '''
>
> [
>     {
>         "role": "system",
>         "content": system_prompt,
>     },
>     {
>         "role": "user",
>         "content": "인터넷이 다시 작동하도록 해야 하는데 어떻게 해야할까?",
>     }
> ]

#### 모델에게 생각을 할 수 있도록 함

##### 스스로 해결책을 생각하도록 지시

결론에 도달하기 전에 근본 원칙에서 사고하도록 명시적으로 지시하면 더 나은 결과를 얻을 수 있음. <br>
&nbsp;&nbsp;&nbsp;&nbsp;ex) 학생의 수학 문제 해결 과정을 평가하고 싶다면, 가장 간단한 방법은 모델에게 학생의 해결책이 맞는지 틀린지 물어보는 방법 등. <br>

<br>

> ```python
> prompt = '''
> 문제 설명: 태양광 설치를 구축 중이며 재무 계산에 도움을 필요 함.
> - 토지 비용: 평방피트당 $100
> - 나는 평방피트당 $250에 태양광 패널을 구입 가능.
> - 나는 연간 고정 비용 $100,000와 평방피트당 추가 비용 $10인 유지보수 계약을 협상.
> 첫 해 운영의 총 비용을 평방피트 수에 대한 함수로 계산할 수 있을까?
> 
> 학생의 솔루션: x를 설치 면적(평방피트)이라고 가정.
> 1. 토지 비용: 100x
> 2. 태양광 패널 비용: 250x
> 3. 유지보수 비용: 100,000 + 100x
> 총 비용: 100x + 250x + 100,000 + 100x = 450x + 100,000
> '''
> 
> [
>     {
>         "role": "system",
>         "content": "학생의 솔루션이 올바른지 여부를 판단해줄래?"
>     },
>     {
>         "role": "user",
>         "content": f"{prompt}"
>     },
> ]
> ```

<br>

학생의 해결책이 실제로는 맞지 않음. <br> 
모델이 이를 성공적으로 인지하도록 하려면, 모델에게 먼저 자신의 해결책을 생성하도록 유도하는 것이 바람직. <br>
-> 모델이 문제를 근본적으로 이해하고, 학생의 해결책과 비교하여 오류를 발견 가능 <br>

<br>

> ```python
> prompt = '''
> 문제 설명: 태양광 설치를 구축 중이며 재무 계산에 도움을 필요 함.
> - 토지 비용: 평방피트당 $100
> - 나는 평방피트당 $250에 태양광 패널을 구입 가능.
> - 나는 연간 고정 비용 $100,000와 평방피트당 추가 비용 $10인 유지보수 계약을 협상.
> 첫 해 운영의 총 비용을 평방피트 수에 대한 함수로 계산할 수 있을까?
> 
> 학생의 솔루션: x를 설치 면적(평방피트)이라고 가정.
> 1. 토지 비용: 100x
> 2. 태양광 패널 비용: 250x
> 3. 유지보수 비용: 100,000 + 100x
> 총 비용: 100x + 250x + 100,000 + 100x = 450x + 100,000
> '''
> 
> [
>     {
>         "role": "system",
>         "content": "먼저 문제를 해결한 후, 자신의 해결책을 학생의 해결책과 비교하고 학생의 해결책이 맞는지 평가해볼래?. 학생의 해결책이 맞는지 결정하기 전에 먼저 문제를 직접 풀어봐야해."
>     },
>     {
>         "role": "user",
>         "content": f"{prompt}"
>     },
> ]
> ```

##### 내부 독백이나 일련의 질문을 사용

일부 애플리케이션에서는 모델이 최종 답변에 도달하기 위해 사용하는 사고 과정을 사용자와 공유하는 것이 부적절할 수 있음. <br>
&nbsp;&nbsp;&nbsp;&nbsp; tutoring app에서는 학생들이 스스로 답을 도출하도록 격려하고 싶지만, 모델의 사고 과정이 학생에게 정답을 드러낼 수 있음 <br>

내부 독백의 아이디어는 모델에게 사용자에게 숨기려는 출력의 일부를 구조화된 형식으로 배치하도록 지시. <br>
-> 출력이 쉽게 분석될 수 있으며, 사용자에게 출력을 제공하기 전에 이 출력을 분석하고, 필요한 부분만 표시하도록 함.

<br>

> ```python
> system_prompt = '''
> 다음 단계에 따라 사용자 문의에 답변.
> 
> 1단계 - 먼저 문제를 스스로 해결해 봐. 학생의 해결책이 틀릴 수 있기에 학생의 해결책에 의존하면 안 돼. 이 단계의 모든 작업은 세 개의 따옴표로 묶어줘 (""").
> 
> 2단계 - 자신의 해결책과 학생의 해결책을 비교하고, 학생의 해결책이 맞는지 평가해. 이 단계의 모든 작업은 세 개의 따옴표로 묶어줘. (""").
> 
> 3단계 - 학생이 실수를 했다면, 정답을 주지 않으면서 학생에게 어떤 힌트를 줄 수 있을지 결정해. 이 단계의 모든 작업은 세 개의 따옴표로 묶어줘. (""").
> 
> 4단계 - 학생이 실수를 했다면, 이전 단계에서 얻은 힌트를 학생에게 제공. (세 개의 따옴표 밖에서). "4단계 - ..." 대신 "힌트:"라고 작성.
> '''
> 
> [
>     {
>         "role": "system",
>         "content": f"{system_prompt}"
>     },
>     {
>         "role": "user",
>         "content": "문제: {problem_statement} \n\n학생 정답: {student_solution}"
>     },
> ]
> ```

<br>

마지막을 제외한 모든 출력이 최종 사용자에게 숨겨진 일련의 질문을 통해 이를 달성 가능. <br>
먼저, 모델에게 문제를 스스로 해결하도록 요청. <br>
&nbsp;&nbsp;&nbsp;&nbsp;이 초기 질문은 학생의 해결책이 필요하지 않으므로 생략 가능. <br>
&nbsp;&nbsp;&nbsp;&nbsp;->모델의 해결책이 학생의 시도한 해결책에 의해 편향될 가능성이 없어짐. <br>

<br>

> ```python
> [
>     {
>         "role": "user",
>         "content": f"{problem_statement}"
>     },
> ]
> ```

<br>

이후 모델이 모든 사용 가능한 정보를 활용하여 학생의 해결책의 정확성을 평가. <br>
이 과정에서 모델은 자신의 해결책과 학생의 해결책을 비교하고, 오류가 있는지 여부를 판단. <br>
이를 통해 학생에게 도움이 되는 피드백을 제공 가능 <br>

<br>

> ```python
> prompt=f'''
> 문제 설명: """{problem_statement}"""
> 당신의 해결책: """{model generated_solution}"""
> 학생의 솔루션: """{students_solution}"""
> '''
>
> [
>     {
>         "role": "system",
>         "content": f"해결책을 학생의 해결책과 비교하고 학생의 해결책이 올바른지 평가."
>     },
>     {
>         "role": "user",
>         "content": f"{prompt}"
>     },
> ]
> ```

<br>

마지막으로, 모델이 자신의 분석을 바탕으로 유익한 튜터의 역할로 답변을 작성하도록 유도. <br>
-> 학생에게 친절하고 이해하기 쉬운 방식으로 피드백을 제공하며, 문제 해결에 대한 도움을 줄 수 있음

<br>

> ```python
> prompt=f'''
> 문제 설명: """{problem_statement}"""
> 당신의 해결책: """{model generated_solution}"""
> 학생의 솔루션: """{students_solution}"""
> 분석: """{model_generated_analysis_from_previous_step}"""
> '''
>
> [
>     {
>         "role": "system",
>         "content": f"당신은 수학 튜터이다. 학생이 오류를 범했다면, 답을 드러내지 않으면서 힌트를 제공. 학생이 오류를 범하지 않았다면, 격려의 말을 전달"
>     },
>     {
>         "role": "user",
>         "content": f"{prompt}"
>     },
> ]
> ```

#### 모델에게 이전 단계에서 놓친 것이 있는지 확인

모델을 사용하여 특정 질문과 관련된 출처에서 발췌된 내용을 나열한다고 가정. <br>
각 발췌 후 모델은 다음 발췌를 작성해야 할지 아니면 멈춰야 할지를 결정해야 함. <br>
만약 출처 문서가 크다면, 모델이 너무 일찍 멈추고 관련된 모든 발췌를 나열하지 못하는 경우가 많음. <br>
이 때, 이전 단계에서 놓친 발췌를 찾기 위해 모델에 후속 질문을 제시하면 더 나은 성과를 얻을 수 있음 <br>

<br>

ex) 첫 번째 쿼리에서 모델이 몇 개의 발췌를 나열한 후, "혹시 더 추가할 만한 발췌가 있어?"와 같은 질문을 통해 모델이 이전에 놓친 내용을 다시 검토하도록 유도. <br>
이 방법은 모델이 보다 포괄적이고 완전한 답변을 제공하도록 함. <br>

<br>

> ```python
> system_prompt=f'''
> 제공된 세 개의 따옴표로 구분된 문서를 사용하여 다음 질문과 관련된 발췌를 선택: "인공지능 역사에서 발생한 중요한 패러다임 전환은 무엇인가?"
> 
> 발췌에는 해석에 필요한 모든 관련 문맥이 포함되어야 해. 즉, 중요한 문맥이 누락된 문장은 추출하지 않아야 해. 
> 출력 형식은 아래와 같아:
> 
> [{"excerpt": "..."},
> ...
> {"excerpt": "..."}]
> '''
>
> [
>     {
>         "role": "system",
>         "content": f"{system_prompt}"
>     },
>     {
>         "role": "user",
>         "content": f"""{documents}"""
>     },
> ]
> ```

<br>

이에 대한 응답이 발생하면 아래와 같이 입력

<br>

> ```python
> [
>     {
>         "role": "user",
>         "content": f"더 관련 있는 발췌가 있어? 기존의 것과 반복되지 않도록 주의해야 해. 또한, 발췌가 해석하는 데 필요한 모든 관련 문맥을 포함하도록 해줘. 즉, 중요한 문맥이 누락된 작은 조각을 추출하지 않도록 해줘"
>     },
> ]
> '''

